In [1]:
import pandas as pd
import numpy as np
import glob
from sklearn import cluster
from scipy.misc import imread
import cv2
import skimage.measure as sm
# import progressbar
import multiprocessing
import random
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
new_style = {'grid': False}
plt.rc('axes', **new_style)

# Function to show 4 images
def show_four(imgs, title):
    #select_imgs = [np.random.choice(imgs) for _ in range(4)]
    select_imgs = [imgs[np.random.choice(len(imgs))] for _ in range(4)]
    _, ax = plt.subplots(1, 4, sharex='col', sharey='row', figsize=(20, 3))
    plt.suptitle(title, size=20)
    for i, img in enumerate(select_imgs):
        ax[i].imshow(img)

# Function to show 8 images
def show_eight(imgs, title):
    select_imgs = [imgs[np.random.choice(len(imgs))] for _ in range(8)]
    _, ax = plt.subplots(2, 4, sharex='col', sharey='row', figsize=(20, 6))
    plt.suptitle(title, size=20)
    for i, img in enumerate(select_imgs):
        ax[i // 4, i % 4].imshow(img)

In [2]:
#select = 500 # Only load 500 images for speed
# Data loading
tr_files = glob.glob('/mnt/kaggle_fisherman/train/*/*.jpg')
te1_files = glob.glob('/mnt/kaggle_fisherman/test_stg1/*.jpg')
te2_files = glob.glob('/mnt/kaggle_fisherman/test_stg2/*.jpg')

In [3]:
files = tr_files + te1_files + te2_files

In [4]:
all_files = np.array([cv2.resize(imread(img), (64, 64), cv2.INTER_LINEAR) for img in files])
print('number of total images {}'.format(len(all_files)))

# Function for computing distance between images
def compare(args):
    img, img2 = args
    img = (img - img.mean()) / img.std()
    img2 = (img2 - img2.mean()) / img2.std()
    return np.mean(np.abs(img - img2))

number of total images 16930


In [ ]:
# Create the distance matrix in a multithreaded fashion
pool = multiprocessing.Pool(12)
#bar = progressbar.ProgressBar(max=len(train))
distances = np.zeros((len(all_files), len(all_files)))
for i, img in enumerate(all_files): #enumerate(bar(train)):
    all_imgs = [(img, f) for f in all_files]
    dists = pool.map(compare, all_imgs)
    distances[i, :] = dists

cls = cluster.DBSCAN(metric='precomputed', min_samples=5, eps=0.6)
boat_ids = cls.fit_predict(distances)

In [7]:
df = pd.DataFrame({'path':files,'boat_id':boat_ids})
df.to_csv('/mnt/kaggle_fisherman/preprocessed_train/boat_ids_train_test_unified.csv')